In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sys.path.append('../..')
import model.Baseline as baseline
import model.my_model as mymodel
import model.util_loss as util_ls
import model.util_dataloader as util_dl
import model.util_model as util_md
import data_process.util_data as util_dt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

# local
file_save_dir = r'C:\Users\29492\Desktop\server_data\sub_ablation'
file_load_dir = r'D:\dataset\DataFrames\REDD'
fig_save_dir = r'C:\Users\29492\Desktop\server_data\plot'
# server
# file_save_dir = r'../../data'
# file_load_dir = r'../../data'

dm = util_md.DictManager(file_save_dir)
mwm = util_md.ModelWeightManager(file_save_dir)

In [ ]:
def averaged_seq(pred_2):
    n_rows, in_seq_l = pred_2.shape
    window_l_half = math.floor((in_seq_l-1)/2)
    pred_2_seq = []
    for i in range(n_rows):
        row_s = max(0, i-window_l_half)
        row_e = min(n_rows-1, i+window_l_half)
        tmp = 0
        num = 0
        for j in range(row_s, row_e+1):
            num +=1
            tmp += pred_2[j,i+window_l_half-j]
        pred_2_seq.append(tmp/num)
    pred_2_seq = np.array(pred_2_seq)
    return pred_2_seq

def median_filter(data, window_size):
    # 确保窗口大小为奇数且不大于数据长度
    if window_size % 2 == 0 or window_size > len(data):
        raise ValueError("Window size must be an odd number and less than or equal to the length of the data.")

    # 对数据进行填充以处理边界情况
    pad_size = window_size // 2
    padded_data = np.pad(data, pad_size, mode='edge')

    # 应用中位数滤波
    filtered_data = []
    for i in range(len(data)):
        window = padded_data[i:i + window_size]
        filtered_data.append(np.median(window))

    return filtered_data

In [ ]:
plt.rcParams['font.family'] = 'Times new Roman'  # Use a serif font family
plt.rcParams['font.size'] = 14

In [ ]:
dataframe_path = os.path.join(file_load_dir, r'house_1.csv')
df = pd.read_csv(dataframe_path)
df.set_index('time', inplace=True)
# df.describe(percentiles=[0.10,0.25,0.50,0.75,0.8,0.85,0.9,0.95])


house_1_cols = ['oven-3', 'oven-4', 'refrigerator-5', 'dishwaser-6', 'lighting-9', 'washer_dryer-10', 'microwave-11',
                'bathroom_gfi-12', 'electric_heat-13', 'stove-14', 'lighting-17', 'lighting-18', 'washer_dryer-19', 'washer_dryer-20']
filtered_columns = df[house_1_cols]
# filtered_columns.describe(percentiles=[0.10,0.25,0.50,0.6,0.75,0.8,0.85,0.9,0.95])


apps_name = filtered_columns.columns.to_list()
print(apps_name)


filtered_columns['sum'] = filtered_columns.sum(axis=1)
norm_data, min_val, max_val = util_dt.normalization_interval(filtered_columns.to_numpy(), 0, 1)
main = norm_data[:,-1]
apps = norm_data[:,:-1]
print(f'min: {min_val}; max: {max_val}')
print(np.min(main), np.max(main))
print(np.min(apps), np.max(apps))
util_dt.print_shape(main, apps)


# on_threshold = [50, 50, 50, 50, 20, 50, 50, 50, 50, 50, 20, 20, 50, 50]
on_threshold = 30
sliding_window_len = 599


w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
w_main = np.expand_dims(w_main, axis=1)
w_apps_sum = w_apps.sum(axis=-1)
w_apps_sum = np.expand_dims(w_apps_sum, axis=1)
w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')

util_dt.print_shape(w_main, w_apps, w_apps_sum, w_apps_on)


in_seq_l = sliding_window_len
out_dim = len(apps_name)
print(f'in_seq_l: {in_seq_l}; out_dim: {out_dim}')


apps_to_train = ['refrigerator-5', 'dishwaser-6', 'washer_dryer-10', 'microwave-11', 'bathroom_gfi-12']
for app in apps_to_train:
    print(apps_name.index(app), app)

In [ ]:
%matplotlib qt

# ablation



In [ ]:
'''
2
refrigerator
    y_max = 700
    y_min = -100
    y_span = 100
    gap = 1
    best so far
    index_s = 78000
    index_e = 82000
3
dishwaser
    y_max = 1900
    y_min = -100
    y_span = 200
    gap = 1
    best so far
    index_s = 54500
    index_e = 54500+450*4
5
washer_dryer
    y_max = 900
    y_min = -100
    y_span = 200
    best so far
    index_s = 56900
    index_e = 56900+450*1
    # index_s = 130700
    # index_e = 130700+450*1
    # index_s = 132600
    # index_e = 132600+450*1
    # index_s = 143300
    # index_e = 143300+450*1
6
microwave
    y_max = 2100
    y_min = -100
    y_span = 200
    best so far
    index_s = 74300
    index_e = index_s+ int(450*0.5)
7
bathroom_gfi
    y_max = 2500
    y_min = -100
    y_span = 200
    best so far
    index_s = 66200
    index_e = index_s+ int(450*0.5)
    time = np.arange(0, index_e-index_s)
'''


In [ ]:
index_app = 2
appname = apps_name[index_app]
print(appname)
index_s = 3700
index_e = 3800

# index_s = 130100
# index_e = index_s+450*1
# index_s = 132500
# index_e = index_s+450*1

time = np.arange(0, index_e-index_s)
time_hours = time * 8 / 3600

tmp_input = w_main[index_s:index_e]
tmp_sum = w_apps_sum[index_s:index_e, 0]*(max_val-min_val)+min_val
tmp_app = w_apps[index_s:index_e, index_app]*(max_val-min_val)+min_val

# fig = plt.figure(figsize=(20, 10))
plt.plot(time, median_filter(tmp_sum,3))
# plt.plot(time, tmp_app, c='blue', label = 'Ground truth', linewidth=1.5)
plt.xlim(0,100)
plt.ylim(100,250)
plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False,
                labelbottom=False, labelleft=False)  # 隐藏刻度
plt.box(on=True)

In [ ]:
index_app = 5
appname = apps_name[index_app]
print(appname)
index_s = 0
index_e = 200000

# index_s = 130100
# index_e = index_s+450*1
index_s = 132500
index_e = index_s+450*1

time = np.arange(0, index_e-index_s)
time_hours = time * 8 / 3600

tmp_input = w_main[index_s:index_e]
tmp_sum = w_apps_sum[index_s:index_e, 0]*(max_val-min_val)+min_val
tmp_app = w_apps[index_s:index_e, index_app]*(max_val-min_val)+min_val

fig = plt.figure(figsize=(20, 10))
plt.plot(time, tmp_sum, c='green', label = 'Input', linewidth=0.5)
plt.plot(time, tmp_app, c='blue', label = 'Ground truth', linewidth=1.5)

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.plot(time, tmp_sum, c='green', label = 'Input', linewidth=0.5)
plt.plot(time, tmp_app, c='blue', label = 'Ground truth', linewidth=1.5)

In [ ]:
tmp_input = w_main[index_s:index_e]
tmp_sum = w_apps_sum[index_s:index_e, 0]*(max_val-min_val)+min_val
tmp_app = w_apps[index_s:index_e, index_app]*(max_val-min_val)+min_val

dataset = util_dl.VariableDataset(torch.from_numpy(tmp_input).to(torch.float32))
dataset.describe()
testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=1000, shuffle_flag=False)

fig_size = (2.5,2.5)
color_GT = 'black'
label_GT = 'Ground truth'
width_GT = 0.5
color_Pred = 'blue'
label_Pred = 'Prediction'
width_Pred = 1
y_max = 1900
y_min = -100
y_span = 400
y_ticks = np.arange(y_min, y_max + y_span, y_span)

In [ ]:

model = mymodel.Model_Mark(in_seq_l, out_dim)
mwm.load_model_weight(model, 'REDD_AblationMark_baseline_all_0103-2')
pred_0 = util_md.predict(device, model, testset)[0][:, index_app]*(max_val-min_val)+min_val

model = mymodel.Model_Mark(in_seq_l, out_dim)
mwm.load_model_weight(model, 'REDD_AblationMark_Ws_all_0103-2')
pred_1 = util_md.predict(device, model, testset)[0][:, index_app]*(max_val-min_val)+min_val

model = mymodel.Model_MTL(in_seq_l, out_dim)
mwm.load_model_weight(model, 'REDD_AblationMark_Ws_Mtl_all_0103-1')
pred_2 = util_md.predict(device, model, testset)[1][:, index_app]*(max_val-min_val)+min_val

model = mymodel.Model_MTL(in_seq_l, out_dim)
mwm.load_model_weight(model, 'REDD_AblationMark_Ws_Mtl_Alw_all_0103-AWL2L-2')
pred_3 = util_md.predict(device, model, testset)[1][:, index_app]*(max_val-min_val)+min_val

model = mymodel.Model_final(in_seq_l, out_dim)
mwm.load_model_weight(model, 'REDD_AblationMark_Ws_Mtl_Alw_Sgns_all_0103-2')
pred_4 = util_md.predict(device, model, testset)[1][:, index_app]*(max_val-min_val)+min_val

data = [pred_0, pred_1, pred_2, pred_3, pred_4]
name = ['baseline', 'Ws','WsMtl','WsMtlAlw', 'WsMtlAlwSgn']
for d, n in zip(data,name):
    fig, ax = plt.subplots(figsize=fig_size)
    ax.plot(time_hours, tmp_app, c=color_GT, label = label_GT)
    ax.plot(time_hours, d, c=color_Pred, label = label_Pred)
    ax.set_ylim(y_min, y_max)
    ax.set_yticks(y_ticks)
    # ax.legend()

In [ ]:
gap = 0.25
xticks = np.arange(min(time_hours), max(time_hours)+gap, gap)
power_ratio = 1e-3
median_filter_setting = 7

In [ ]:
data = [pred_0, pred_1, pred_2, pred_3, pred_4]
name = ['baseline', 'Ws','WsMtl','WsMtlAlw', 'WsMtlAlwSgn']
gt = [element*power_ratio for element in median_filter(tmp_app,median_filter_setting)]
for d, n in zip(data,name):
    pred = [element*power_ratio for element in median_filter(d,median_filter_setting)]
    fig, ax = plt.subplots(figsize=fig_size)
    ax.plot(time_hours, gt, c=color_GT, label = label_GT)
    ax.plot(time_hours, pred, c=color_Pred, label = label_Pred)
    ax.set_ylim(y_min*power_ratio, y_max*power_ratio)
    ax.set_yticks(y_ticks*power_ratio)
    ax.set_xticks(xticks)
    # ax.legend(loc='upper center')
    # ax.set_xlabel('Time (hour)')
    # ax.set_ylabel('Power (Kilo Watt)')
    fig.savefig(os.path.join(fig_save_dir, f'sub_ablation_{appname}_{n}.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=fig_size)
ax.plot(time_hours, [element*power_ratio for element in median_filter(tmp_sum,median_filter_setting)], c='green', label = 'Input', linewidth=0.5)
ax.plot(time_hours, [element*power_ratio for element in median_filter(tmp_app,median_filter_setting)], c=color_GT, label = label_GT, linewidth=1.5)
# ax.legend()
ax.set_ylim(y_min*power_ratio, y_max*power_ratio)
ax.set_yticks(y_ticks*power_ratio)
ax.set_xticks(xticks)
# ax.set_ylim(0, 4)
# ax.legend(loc='upper center')
# ax.set_xlabel('Time (hour)')
# ax.set_ylabel('Power (Kilo Watt)')
ax.get_tightbbox()
fig.savefig(os.path.join(fig_save_dir, f'sub_ablation_{appname}_GT.pdf'), format="pdf", bbox_inches='tight')

# feasibility

In [ ]:
index_app = 3
appname = apps_name[index_app]
print(appname)
index_s = 0
index_e = 200000
index_s = 54500
index_e = 54500+450*4
time = np.arange(0, index_e-index_s)
time_hours = time * 8 / 3600

tmp_input = w_main[index_s:index_e]
tmp_sum = w_apps_sum[index_s:index_e, 0]*(max_val-min_val)+min_val
tmp_app = w_apps[index_s:index_e, index_app]*(max_val-min_val)+min_val

fig = plt.figure(figsize=(20, 10))
plt.plot(time, tmp_sum, c='green', label = 'Input', linewidth=0.5)
plt.plot(time, tmp_app, c='blue', label = 'Ground truth', linewidth=1.5)

In [ ]:
tmp_input = w_main[index_s:index_e]
tmp_sum = w_apps_sum[index_s:index_e, 0]*(max_val-min_val)+min_val
tmp_app = w_apps[index_s:index_e, index_app]*(max_val-min_val)+min_val

dataset = util_dl.VariableDataset(torch.from_numpy(tmp_input).to(torch.float32))
dataset.describe()
testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=1000, shuffle_flag=False)


In [ ]:
fig_size = (3,3)
color_GT = 'black'
label_GT = 'Ground truth'
width_GT = 0.5
color_Pred = 'blue'
label_Pred = 'Prediction'
width_Pred = 1
y_max = 2300
y_min = -100
y_span = 400
y_ticks = np.arange(y_min, y_max + y_span, y_span)

In [ ]:

model = mymodel.Model_Mark(in_seq_l, out_dim)
mwm.load_model_weight(model, 'REDD_AblationMark_baseline_all_0103-0')
pred_0 = util_md.predict(device, model, testset)[0][:, index_app]*(max_val-min_val)+min_val

model = mymodel.Model_Mark(in_seq_l, out_dim)
mwm.load_model_weight(model, 'REDD_AblationMark_Ws_all_0103-2')
pred_1 = util_md.predict(device, model, testset)[0][:, index_app]*(max_val-min_val)+min_val

data = [pred_0, pred_1]
name = ['Ss', 'Ws']
for d, n in zip(data,name):
    fig, ax = plt.subplots(figsize=fig_size)
    ax.plot(time_hours, tmp_app, c=color_GT, label = label_GT, linewidth=width_GT)
    ax.plot(time_hours, d, c=color_Pred, label = label_Pred, linewidth=width_Pred)
    ax.set_ylim(y_min, y_max)
    ax.set_yticks(y_ticks)
    ax.legend(loc='upper right')
    # fig.savefig(os.path.join(fig_save_dir, f'sub_ablation_{n}.png'), dpi=1000, bbox_inches='tight')

In [ ]:
gap = 1
xticks = np.arange(min(time_hours), max(time_hours)+gap, gap)
power_ratio = 1
median_filter_setting = 7

In [ ]:
data = [pred_0, pred_1]
name = ['Ss', 'Ws']
gt = [element*power_ratio for element in median_filter(tmp_app,median_filter_setting)]
for d, n in zip(data,name):
    pred = [element*power_ratio for element in median_filter(d,median_filter_setting)]
    fig, ax = plt.subplots(figsize=fig_size)
    ax.plot(time_hours, gt, c=color_GT, label = label_GT)
    ax.plot(time_hours, pred, c=color_Pred, label = label_Pred)
    ax.set_ylim(y_min*power_ratio, y_max*power_ratio)
    ax.set_yticks(y_ticks*power_ratio)
    ax.set_xticks(xticks)
    # ax.legend(loc='upper right')
    ax.set_xlabel('Time (hour)')
    ax.set_ylabel('Power (Watt)')
    # fig.savefig(os.path.join(fig_save_dir, f'feasibility_{appname}_{n}.png'), dpi=1000, bbox_inches='tight')
    fig.savefig(os.path.join(fig_save_dir, f'feasibility_{appname}_{n}.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=fig_size)
ax.plot(time_hours, [element*power_ratio for element in median_filter(tmp_sum,median_filter_setting)], c='green', label = 'Input')
ax.plot(time_hours, [element*power_ratio for element in median_filter(tmp_app,median_filter_setting)], c=color_GT, label = label_GT)
ax.set_ylim(y_min*power_ratio, y_max*power_ratio)
ax.set_yticks(y_ticks*power_ratio)
ax.set_xticks(xticks)
# ax.set_ylim(0, 4)
# ax.legend()
# ax.legend(loc='upper right')
ax.set_xlabel('Time (hour)')
ax.set_ylabel('Power (Kilo Watt)')
ax.get_tightbbox()
fig.savefig(os.path.join(fig_save_dir, f'feasibility_{appname}_GT.pdf'), format="pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.plot(np.arange(10), c='green', label = 'Input')
ax.plot(np.arange(10)+1, c=color_GT, label = label_GT)
ax.plot(np.arange(10)+2, c=color_Pred, label = label_Pred)
ax.set_ylim(0,50)
# ax.set_ylim(0, 4)
ax.legend(loc = 'center', ncol=3)

# fig.legend(loc=loc, title=loc)
ax.set_xlabel('Time (hour)')
ax.set_ylabel('Power (Kilo Watt)')
ax.get_tightbbox()
fig.savefig(os.path.join(fig_save_dir, f'labels.pdf'), format="pdf", bbox_inches='tight')